In [ ]:
import os
import torch
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Configuration
MAX_CHARS = 1024
MAX_FEATURES = 3072
SEED = 42
torch.manual_seed(SEED)

In [ ]:
# Custom PyTorch Dataset
class EmailDataset(Dataset):
    def __init__(self, folder, label):
        self.texts = []
        self.labels = []
        self.paths = []
        for filename in os.listdir(folder):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()[:MAX_CHARS]
                self.texts.append(text)
                self.labels.append(label)
                self.paths.append(filepath)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx], self.paths[idx]

In [ ]:
# Define paths
# negative_folder = "enron/kaminski-nyt" # llm generated
# negative_folder = "enron/top11-o" # 11 enron senders
negative_folder = "enron/BEC-2-emails" # BEC-2 + llm generated + 11 enron senders 
# negative_folder = "enron/nyt-alt" # BEC-2 + 11 enron senders
# positive_folder = "enron/kaminski-v" # kaminski-v sender
# positive_folder = "enron/kaminski-nyt"
positive_folder = "enron/stclair-c" # stclair-c sender

In [ ]:
neg_dataset = EmailDataset(negative_folder, 0)
pos_dataset = EmailDataset(positive_folder, 1)

all_texts = neg_dataset.texts + pos_dataset.texts
all_labels = neg_dataset.labels + pos_dataset.labels
all_paths = neg_dataset.paths + pos_dataset.paths

In [ ]:
# Combined Dataset
class CombinedDataset(Dataset):
    def __init__(self, texts, labels, paths):
        self.texts = texts
        self.labels = labels
        self.paths = paths

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx], self.paths[idx]

dataset = CombinedDataset(all_texts, all_labels, all_paths)

In [ ]:
# Random train/test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

train_texts = [x[0] for x in train_set]
train_labels = [x[1] for x in train_set]
train_paths = [x[2] for x in train_set]
test_texts = [x[0] for x in test_set]
test_labels = [x[1] for x in test_set]
test_paths = [x[2] for x in test_set]

In [ ]:
# Count vectorization
vectorizer = CountVectorizer(lowercase=False, stop_words='english', max_features=MAX_FEATURES)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [ ]:
# Naive Bayes classifier
model = MultinomialNB(alpha=1.0)
model.fit(X_train, train_labels)

In [ ]:
# Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, average='macro')

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# Identify false predictions
false_positives = []
false_negatives = []

for true_label, pred_label, file_path in zip(test_labels, y_pred, test_paths):
    if true_label == 0 and pred_label == 1:
        false_positives.append(file_path)
    elif true_label == 1 and pred_label == 0:
        false_negatives.append(file_path)

print("\nFalse Positives:")
for path in false_positives:
    print(f"  {path}")

print("\nFalse Negatives:")
for path in false_negatives:
    print(f"  {path}")

In [ ]:
# Display the confusion matrix
cm_labels = np.array([1, 0])
ConfusionMatrixDisplay.from_predictions(test_labels, y_pred, colorbar=False, labels=cm_labels, cmap='binary')
plt.show()

In [ ]:
# Find Top 10 Most Useful Words in Naive Bayes Classification

# Get feature names from the CountVectorizer
feature_names = vectorizer.get_feature_names_out()

# Get log probabilities of features for each class
log_probs = model.feature_log_prob_  # shape: [n_classes, n_features]

# Compute difference in log probs between classes (positive - negative)
# If class 1 = positive, class 0 = negative
log_prob_diff = log_probs[1] - log_probs[0]

# Get indices of top 10 most indicative words for positive class
top_pos_indices = np.argsort(log_prob_diff)[-10:][::-1]
top_neg_indices = np.argsort(log_prob_diff)[:10]

# Get top words
top_positive_words = [(feature_names[i], log_prob_diff[i]) for i in top_pos_indices]
top_negative_words = [(feature_names[i], log_prob_diff[i]) for i in top_neg_indices]

print("Top 10 Positive Class Words:")
for word, score in top_positive_words:
    print(f"{word:20} {score:.4f}")

print("\nTop 10 Negative Class Words:")
for word, score in top_negative_words:
    print(f"{word:20} {score:.4f}")

In [ ]:
# Extract word names and scores from top_* lists
positive_words = [word for word, _ in top_positive_words]
positive_scores = [score for _, score in top_positive_words]

negative_words = [word for word, _ in top_negative_words]
negative_scores = [score for _, score in top_negative_words]

# Combine word labels with class tags and absolute scores
combined_words = [f"+ {w}" for w in positive_words] + [f"- {w}" for w in negative_words]
combined_scores = [abs(s) for s in positive_scores + negative_scores]

# Sort by informativeness (descending)
sorted_indices = np.argsort(combined_scores)[::-1]
combined_words = [combined_words[i] for i in sorted_indices]
combined_scores = [combined_scores[i] for i in sorted_indices]

# Grayscale colors
gray_colors = plt.cm.Greys(np.linspace(0.4, 0.9, len(combined_scores)))

# Plot (taller and narrower)
plt.figure(figsize=(8, 8))  # Adjust for two-column fit
bars = plt.barh(combined_words[::-1], combined_scores[::-1], color=gray_colors[::-1])
plt.xlabel("Informativeness (|Log Probability Difference|)")
# plt.title("Most Informative Words (Naive Bayes)", fontsize=11)o

# Set font size for the Y-axis labels
plt.yticks(fontsize=11)  # Adjust this value as needed

# Annotate each bar
for bar in bars:
    width = bar.get_width()
    plt.text(
        width + 0.01,
        bar.get_y() + bar.get_height() / 2,
        f"{width:.2f}",
        va='center',
        ha='left',
        fontsize=8,
        color='black'
    )

plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.show()